# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1613469734.0
Main triggered 1613469734.5
Main triggered 1613469735.0
Print triggered 1613469735.0
Main triggered 1613469735.5


# Controller Base Class (eFMU)

In [6]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 30, 'b': 22}
Output {'c': 660}
Output {'c': 660}


# Controller Stack Class (single-thread/multi-thread)

In [26]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-02-16 02:43:27.546515968,NaN,NaN,NaN,Initialize
2021-02-16 02:44:13.794136064,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-16 02:44:16.061634048,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-16 02:44:18.319866880,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-16 02:44:20.585547008,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-16 02:44:22.855887104,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-16 02:44:25.116613120,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-16 02:43:27.546515968,NaN,NaN,NaN,Initialize
2021-02-16 02:44:13.794136064,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-16 02:44:16.061634048,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-16 02:44:18.319866880,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-16 02:44:20.585547008,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-16 02:44:22.855887104,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-16 02:44:25.116613120,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-16 02:43:27.546515968,NaN,NaN,NaN,Initialize
2021-02-16 02:44:13.794136064,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-16 02:44:16.061634048,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-16 02:44:18.319866880,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-16 02:44:20.585547008,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-16 02:44:22.855887104,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-16 02:44:25.116613120,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-16 02:43:27.546515968,NaN,NaN,NaN,Initialize
2021-02-16 02:44:13.794136064,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-16 02:44:16.061634048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-16 02:44:18.319866880,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-16 02:44:20.585547008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-16 02:44:22.855887104,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-16 02:44:25.116613120,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-16 02:43:27.546515968,NaN,NaN,NaN,Initialize
2021-02-16 02:44:13.794136064,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-16 02:44:16.061634048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-16 02:44:18.319866880,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-16 02:44:20.585547008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-16 02:44:22.855887104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-16 02:44:25.116613120,30.0,4.0,120.0,testcontroller1 did a computation!


13.684929847717285


## Multi Thread

In [3]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:10:56.697136128,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-18 17:10:57.947060992,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-18 17:10:59.198970112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-18 17:11:00.455638016,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-18 17:11:01.706215936,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-18 17:11:02.969302016,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:10:56.697136128,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-18 17:10:57.947060992,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-18 17:10:59.198970112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-18 17:11:00.455638016,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-18 17:11:01.706215936,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-18 17:11:02.969302016,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:10:56.697136128,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-18 17:10:57.947060992,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-18 17:10:59.198970112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-18 17:11:00.455638016,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-18 17:11:01.706215936,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-18 17:11:02.969302016,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:10:56.697136128,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-18 17:10:57.947060992,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-18 17:10:59.198970112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-18 17:11:00.455638016,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-18 17:11:01.706215936,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-18 17:11:02.969302016,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:10:56.697136128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-18 17:10:57.947060992,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-18 17:10:59.198970112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-18 17:11:00.455638016,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-18 17:11:01.706215936,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-18 17:11:02.969302016,30.0,4.0,120.0,testcontroller1 did a computation!


7.635985374450684


In [10]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
threads = []
for i in range(6):
    threads.append(threading.Thread(target=ctrl_stack.query_control, args=[time.time()]))
    time.sleep(.1)
    threads[-1].start()

time.sleep(20)
for thread in threads:
    thread.join()
    if thread.is_alive():
        print(i)
        thread.terminate()
        print('Controller timeout', thread_name)
print(time.time() - t)

20.620548009872437


In [9]:
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:14:02.335298048,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:14:02.335298048,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:14:02.335298048,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:14:02.335298048,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-18 17:10:43.298074880,NaN,NaN,NaN,Initialize
2021-02-18 17:14:02.335298048,30.0,4.0,120.0,testcontroller1 did a computation!
